In [ ]:
from parinterp import Linear2DInterpolator
import numpy as np
import os
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from scipy.interpolate import griddata

In [ ]:
arrays = {}

for item in list(os.walk('/homes/lbelkov/griddata/'))[1:]:
    arrays[item[0][2:][len('omes/lbelkov/griddata/'):]] = np.load(item[0] + '/' + item[2][0], allow_pickle=True)
ids = tuple(arrays.keys())

In [ ]:
for k, x in tqdm(arrays.items()):
    add_cols = x.shape[1] // 4
    distances = np.concatenate((x[...,-add_cols:], x, x[...,:add_cols] ), axis=1)
    int_points = np.transpose((np.isnan(distances)).nonzero())
    x_points: np.ndarray = np.transpose((~np.isnan(distances)).nonzero())
    x_values = distances[~np.isnan(distances)]
    parinterp_map = distances.copy()
    scipy_map = distances.copy()
    parinterp_values = Linear2DInterpolator(x_points, -1)(int_points, x_values, fill_value=0.0)
    scipy_values = griddata(x_points, x_values, int_points, method='linear', fill_value=0.0)

    for i, v in enumerate(parinterp_values):
        parinterp_map[int_points[i][0], int_points[i][1]] = v

    for i, v in enumerate(scipy_values):
        scipy_map[int_points[i][0], int_points[i][1]] = v

    delta = np.abs(parinterp_map - scipy_map)
    q99_delta = np.quantile(delta, 0.99)
    max_delta = np.max(delta)
    outliers_num = np.sum(delta > q99_delta)

    fig, ax = plt.subplots(nrows=1, ncols=2)
    # fig.set_figheight(50)
    fig.set_figwidth(20)
    print('interpolated points shape', int_points.shape)
    print('q99 delta =', q99_delta)
    print('max delta =', max_delta)
    print('num of points where delta > q99_delta =', outliers_num)
    # ax[0].imshow(parinterp_map)
    # ax[0].set_title('parinterp')
    # ax[1].imshow(scipy_map)
    # ax[1].set_title('scipy.interpolate.griddata')
    # ax[2].imshow(delta / scipy_map)
    # ax[2].set_title('delta normed by scipy_map')
    q = 99
    top_delta = np.sort(delta.ravel())[int(delta.size * q / 100):]
    ax[0].hist(top_delta, bins=32)
    ax[0].set_title(f'> q{q} delta')
    ax[1].hist((top_delta / max_delta).ravel(), bins=32)
    ax[1].set_title(f'> q{q} / max_delta')
    plt.show()